In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.shape

(404290, 6)

In [5]:
new_df = df.sample(50000)

In [6]:
new_df.sample(10)

,id,qid1,qid2,question1,question2,is_duplicate
211007,211007,241633,30154,What individuals and events in history are a s...,What individuals and events in history are a s...,0
212510,212510,317614,317615,What is your favorite Gandhi quote?,What is your favorite M. K. Gandhi quote?,1
248975,248975,362499,362500,Who won the Nobel Peace Prize in 2014?,Should Malala Yousafzai have won the 2013 Nobe...,0
40644,40644,73498,73499,Why did you convert to islam?,Why did you convert to Islam as an adult?,1
161973,161973,252244,139941,Which are the best books for CA final?,What are the best books of CA final?,1
299381,299381,422023,422024,Can mechanical engineers do the work of chemic...,Can mechanical engineers work as electrical en...,0
337055,337055,13094,464485,Did Trump land the DC post office project by f...,Do you think most of today's top artists will ...,0
311311,311311,435567,435568,Is it recommendable to do LLM after LLB for be...,What is the career scope after CS with LLB? Al...,0
285170,285170,405539,405540,Is there career for Mechanical engineers in su...,What is the career prospects in thermal power ...,0
84684,84684,143145,143146,What is the best position for sex?,What is the Best position for a quick sex?,0


In [7]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

As there are no duplicates, we can go forward without any deletion of rows.

In [8]:
new_df.duplicated().sum()

0

In [9]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
300798,How are the brakes of tvs Jupiter base model c...,How does it feel like to own a TVS Jupiter?
402969,How are the metro trains able to consistently ...,How are metro trains able to stop at the very ...
203095,Why is YouTube not working on my computer and ...,Why are Youtube videos not playing? How can I ...
109494,Are there any cockpit-to-cockpit frequencies o...,Does Lufthansa allow cockpit visits?
109536,How can I download flash season 3 episode 2?,From where to download episode2 season 3 of Th...


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [11]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(50000, 6000)

In [15]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [16]:
temp_df


,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
300798,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
402969,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
203095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
109494,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
109536,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212378,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
290764,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
191462,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29946,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7635

In [20]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7384